<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Library
## Example: STILT

This example is showcasing how to **search** for STILT stations

## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/), source is available on [github](https://github.com/ICOS-Carbon-Portal/pylib)

## Load the library

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime as dt
from icoscp.stilt import stiltstation

In [ ]:
# create a simple function to prit the Station name
def stprint(stations):
    if not 'empty' in stations.keys():
        for s in stations.keys():
            print(stations[s]['id'],stations[s]['name'], stations[s]['years'])
    else:
        print(stations)

## Help

In [ ]:
help(stiltstation)

## Find stations:


### arbitrary text search

In [ ]:
myStations = stiltstation.find(search='KIT')
stprint(myStations)

### by country

In [ ]:
myStations = stiltstation.find(country='poland')
stprint(myStations)

### by ID

In [ ]:
myStations = stiltstation.find(id='TA')
stprint(myStations)

In [ ]:
myStations = stiltstation.find(id='KIT')
stprint(myStations)

### spatial filters

#### bounding box
approximately scandinavia

In [ ]:
myStations = stiltstation.find(bbox=[(70,5),(55,32)]) 
stprint(myStations)

#### pinpoint
approximately Lund with 200 km distance.

In [ ]:
myStations = stiltstation.find(pinpoint=[55.7,13.1,200]) 
stprint(myStations)

### temporal
Please be aware, that the actual data is not filtered, the result will return stations where data is **available** according to your filter. Three keywords are available, **sdate**: >= start date, **edate**: <= end date, **>dates**: [ ], a list of dates, NOT a range. If you provide sdate AND edate, stations are returned where data is available within that range.
<br> The following input formats are accepted:
- string: yyyy yyyy-mm yyyy/mm yyyy-mm-dd yyyy/mm/dd 
- float or integer representing a unixtimestamp for seconds since 1970.01.01
- python datetime object
- pandas datetime object

#### start date

In [ ]:
myStations = stiltstation.find(sdate='2020/05')
stprint(myStations)

#### end date

In [ ]:
import datetime
myStations = stiltstation.find(edate='2006-12-31')
stprint(myStations)

In [ ]:
# Using a UNIX Timestamp (seconds sincs 1970-01-01)
import time
now = time.time()
print(now)
myStations = stiltstation.find(edate=now)
len(myStations)

#### range

In [ ]:
myStations = stiltstation.find(sdate= '2018-05-01', edate='2018-06-01')
for s in myStations:
    print(myStations[s]['id'], sorted(myStations[s]['years']))

In [ ]:
myStations = stiltstation.find(sdate= '2018', edate='2018')
stprint(myStations)

#### project
The choice `project = 'icos'` will only return STILT-stations that are ICOS-stations.

In [ ]:
myStations = stiltstation.find(project = 'icos')
stprint(myStations)

### combine filters
Filters are applied in sequence of input and each result is the base for the next filter.
<br>For example you could search for stations in a bounding box and filter the result for a specific country.

In [ ]:
myStations = stiltstation.find(bbox=[(70,5),(55,32)], country='Norway') 
stprint(myStations)

In [ ]:
myStations = stiltstation.find(sdate= '2018-05-01', edate='2018-06-01', country='IT')
for s in myStations:
    print(myStations[s]['id'], sorted(myStations[s]['years']))

## Output format
### dict (default)

In [ ]:
stiltstation.find(id='HUN115', outfmt='dict')

### pandas

In [ ]:
stiltstation.find(country=['hungary','austria'], outfmt='pandas')

### map (folium, leaflet)
Display the map directly if you are working in a Jupyter Notebook, 
<br> or save the map to html file if you work in python.

In [ ]:
stiltstation.find(country='italy' ,outfmt='map')

#### save the map

In [ ]:
# define the output folder
output_directory = os.path.join(os.path.expanduser('~'), 'output/')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# this is the actual command to save the map
stiltstation.find(country='finland' ,outfmt='map').save(output_directory + 'myMap.html')

### Availability of data
The flag `outfmt = 'avail'` returns a pandas dataframe with information about the availability of data over time.

In [ ]:
df = stiltstation.find(outfmt = 'avail')

# We can save into a csv-file (first we repeat
# the code of the cell above, in case you did not run it:
output_directory = os.path.join(os.path.expanduser('~'), 'output/')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
df.to_csv(output_directory + 'Stilt_availability_' + dt.datetime.now().strftime('%Y%m%d') + '.csv')

df

#### Better view of the availability table
To get better overview we color the values using pandas. <br>
For further reading see: https://pandas.pydata.org/pandas-docs/version/1.1/user_guide/style.html

In [ ]:
def background_gradient(colormap = 'Blues', m =-5, M=17):
    # This function will just run once to produce a list of 13 colors   
    #     - colormap 'Blues' is a gradient color in blue
    #       m = -5 means very light blue, M = 17 means dark blue

    norm = colors.Normalize(m,M)
    normed = norm(range(13))      
    month_hexcolors = [colors.rgb2hex(x) for x in plt.cm.get_cmap(colormap)(normed)]
    return month_hexcolors

def cell_style(column, palette):
    return ['text-align:center; background-color: %s' % palette[val] for val in column]
    
color_palette = background_gradient()
    
# Coloring cells and center the values of the dataframe 
# (The return df_styled is actually a 'Styler' (pandas.io.formats.style.Styler)
df_styled = df.style.apply(cell_style, subset = df.columns[1:-1], palette = color_palette) 

# Also, we might like a title 
df_styled.set_caption('<center style="font-size:16px"> <b>STILT availability</b></center>')